In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.utils.data import DataLoader, Dataset
from itertools import product

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Kathmandu-Precipitation/data/Outlier-removed-dataset.csv")
features = [ i for i in df.columns if i not in ["precipitation","datetime"]]
target = ["precipitation"]
X = df[features]
y = df[target]

In [ ]:
scaler_features = StandardScaler()
scaler_target = StandardScaler()
features_scaled = scaler_features.fit_transform(X)
target_scaled = scaler_target.fit_transform(y).flatten()

(1707, 7)
(1707, 1)


In [ ]:
# Create sequences for LSTM
def create_sequences(data, target, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = target[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10  # Number of time steps in each sequence
X, y = create_sequences(features_scaled, target_scaled, seq_length)

In [ ]:
# Split into training, validation, and testing sets (rolling window)
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
test_size = len(X) - train_size - val_size

X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Hyperparameter grid
param_grid = {
    'hidden_size': [32, 64],
    'num_layers': [1, 2],
    'learning_rate': [0.001, 0.01]
}

best_rmse = float('inf')
best_params = None

for params in product(*param_grid.values()):
    hidden_size, num_layers, learning_rate = params

    # Instantiate the model
    model = LSTMModel(input_size=7, hidden_size=hidden_size, num_layers=num_layers, output_size=1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    num_epochs = 20
    batch_size = 32
    train_data = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print training loss
        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    # Evaluate on validation set
    model.eval()
    with torch.no_grad():
        val_predictions = model(X_val).squeeze().numpy()
        val_predictions_rescaled = scaler_target.inverse_transform(val_predictions.reshape(-1, 1)).flatten()
        y_val_rescaled = scaler_target.inverse_transform(y_val.numpy().reshape(-1, 1)).flatten()

        val_mse = mean_squared_error(y_val_rescaled, val_predictions_rescaled)
        val_rmse = np.sqrt(val_mse)

    print(f"Params: {params}, Validation MSE: {val_mse:.4f}, Validation RMSE: {val_rmse:.4f}")

    # Save best parameters
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_params = params

print(f"Best Params: {best_params}, Best Validation RMSE: {best_rmse:.4f}")

Epoch [1/20], Training Loss: 0.8834
Epoch [2/20], Training Loss: 0.8290
Epoch [3/20], Training Loss: 0.8275
Epoch [4/20], Training Loss: 0.8249
Epoch [5/20], Training Loss: 0.8235
Epoch [6/20], Training Loss: 0.8224
Epoch [7/20], Training Loss: 0.8213
Epoch [8/20], Training Loss: 0.8203
Epoch [9/20], Training Loss: 0.8194
Epoch [10/20], Training Loss: 0.8184
Epoch [11/20], Training Loss: 0.8175
Epoch [12/20], Training Loss: 0.8164
Epoch [13/20], Training Loss: 0.8153
Epoch [14/20], Training Loss: 0.8140
Epoch [15/20], Training Loss: 0.8127
Epoch [16/20], Training Loss: 0.8112
Epoch [17/20], Training Loss: 0.8097
Epoch [18/20], Training Loss: 0.8082
Epoch [19/20], Training Loss: 0.8064
Epoch [20/20], Training Loss: 0.8042
Params: (32, 1, 0.001), Validation MSE: 169.6379, Validation RMSE: 13.0245
Epoch [1/20], Training Loss: 0.8489
Epoch [2/20], Training Loss: 0.8401
Epoch [3/20], Training Loss: 0.8379
Epoch [4/20], Training Loss: 0.8336
Epoch [5/20], Training Loss: 0.8277
Epoch [6/20], 

In [ ]:
# Retrain with best parameters
hidden_size, num_layers, learning_rate = best_params
model = LSTMModel(input_size=7, hidden_size=hidden_size, num_layers=num_layers, output_size=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Combine training and validation sets
X_train_full = torch.cat((X_train, X_val), dim=0)
y_train_full = torch.cat((y_train, y_val), dim=0)
train_data = torch.utils.data.TensorDataset(X_train_full, y_train_full)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_loader):.4f}")

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test).squeeze().numpy()
    test_predictions_rescaled = scaler_target.inverse_transform(test_predictions.reshape(-1, 1)).flatten()
    y_test_rescaled = scaler_target.inverse_transform(y_test.numpy().reshape(-1, 1)).flatten()

    test_mse = mean_squared_error(y_test_rescaled, test_predictions_rescaled)
    test_rmse = np.sqrt(test_mse)
    test_mae = mean_absolute_error(y_test_rescaled, test_predictions_rescaled)

print(f"Test Metrics:")
print(f"  Test MSE: {test_mse:.4f}")
print(f"  Test RMSE: {test_rmse:.4f}")
print(f"  Test MAE: {test_mae:.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/20], Training Loss: 0.8564


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/20], Training Loss: 0.8117
Epoch [3/20], Training Loss: 0.8093
Epoch [4/20], Training Loss: 0.8066
Epoch [5/20], Training Loss: 0.8009


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch [16/20], Training Loss: 0.7697


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [17/20], Training Loss: 0.7413
Epoch [18/20], Training Loss: 0.7039


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [19/20], Training Loss: 0.7132
Epoch [20/20], Training Loss: 0.7108
Test Metrics:
  Test MSE: 208.9143
  Test RMSE: 14.4539
  Test MAE: 7.4160


In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze().numpy()
    predictions_rescaled = scaler_target.inverse_transform(predictions.reshape(-1, 1)).flatten()
    y_test_rescaled = scaler_target.inverse_transform(y_test.numpy().reshape(-1, 1)).flatten()

# Calculate evaluation metrics (e.g., RMSE)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test_rescaled, predictions_rescaled))
print(f"Test RMSE: {rmse:.4f}")

Test RMSE: 15.0719
